## 1. 라이브러리 및 기업리스트 데이터 불러오기

In [128]:
import pandas as pd
import urllib.request
import json
from tqdm.notebook import tqdm
import time
import requests
from bs4 import BeautifulSoup as bs
import warnings
warnings.filterwarnings('ignore')

In [3]:
kosdaq = pd.read_excel("C:/Users/hanah/Desktop/workspace/Project/ESG/crawling/dart/kosdaq_report_2021.xlsx")
kosdaq

,Unnamed: 0,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,...,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,사업자등록번호,감사의견_강조사항,감사의견_핵심감사사항
0,1508855,369370,블리츠웨이,"인형,장난감 및 오락용품 제조업",기업 인수 및 합병,2020-12-23,12월,배성웅,NaN,서울특별시,...,"2,621","2,628",99.73%,"5,824,093","35,615,566",16.35%,적정의견,6408701749,주1),주2)
1,921916,357550,석경에이티,기초 화학물질 제조업,바이오 및 전기전자 나노 소재,2020-12-23,12월,임형섭,http://www.sukgyung.com,경기도,...,"13,983","13,988",99.96%,"3,036,069","5,455,000",55.66%,적정,1348163122,해당사항 없음,해당사항 없음
2,1276026,314130,지놈앤컴퍼니,기초 의약물질 및 생물학적 제제 제조업,"마이크로바이옴 기반 치료제, 화장품, 건강기능식품",2020-12-23,12월,"배지수, 박한수, 서영진 (각자 대표이사)",http://genomecom.co.kr,경기도,...,"24,231","24,254",99.91%,"6,219,073","14,066,271",44.21%,적정,6278700257,-,"주식보상비용, 사업결합"
3,1137295,335810,프리시젼바이오,의료용 기기 제조업,체외진단 기기 및 시약,2020-12-22,12월,김한신,http://precision-bio.com/,대전광역시,...,"19,462","19,471",99.9%,"6,652,727","11,366,060",58.54%,적정,3148608943,-,제품 및 상품 판매로 인한 수익인식
4,1405451,347860,알체라,소프트웨어 개발 및 공급업,"얼굴인식 AI, 이상상황 감지 AI 등",2020-12-21,12월,황영규,http://www.alchera.ai,경기도,...,"64,032","64,037",99.99%,"8,007,354","13,720,283",58.36%,적정,6438700337,-,수익인식기준
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,1989-12-15,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,...,"9,787","9,804",99.83%,"4,514,128","12,856,050",35.11%,적정,6038106103,해당사항 없음,특수관계자 거래의 내부거래 제거 적정성
1384,156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,1989-09-06,12월,공동대표이사 이준효 공동대표이사 소우에이이치로,http://www.sbik.co.kr,서울특별시,...,"47,521","47,523",99.99%,"90,462,132","162,066,575",55.82%,적정,1208164573,해당사항 없음,수준3 금융상품공정가치
1385,110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,1989-03-14,12월,김형준,http://www.geminivc.co.kr,서울특별시,...,"9,805","9,816",99.88%,"49,318,593","97,953,218",50.35%,적정,1168116053,-,"연결재무제표에 대한 주석10과 25에서 기술한 바와 같이, 회사는 2021년 3월 ..."
1386,136642,19590,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,1989-03-14,12월,홍성혁,http://www.m-vc.co.kr,서울특별시,...,"4,227","4,243",99.62%,"44,146,685","78,398,843",56.31%,적정,1168115961,-,별도 : 당기손익-공정가치측정금융자산의 공정가치 평가\n연결 : 당기손익-공정가치측...


## 2. 네이버 API로 잡플래닛 url 수집

In [16]:
client_id = "k1g3Zx8nVTbbRmXRx3qS"
client_secret = "wzwAGiJ2md"
headers = {'user-agent': 'Mozilla/5.0'} 

jobplanet = pd.DataFrame(columns=('회사명', '링크'))
for idx in tqdm(range(len(kosdaq))):    
    encText = urllib.parse.quote(f'잡플래닛 {kosdaq.회사명[idx]}')
    url = "https://openapi.naver.com/v1/search/webkr?query="+encText +"&display=2"
    time.sleep(0.1)
    request = urllib.request.Request(url, headers=headers)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        try:
            items = response_dict['items'] 
            for i in range(len(items)):
                if 'https://www.jobplanet.co.kr/companies/' in items[i]['link']:
                    company = kosdaq.회사명[idx]
                    link = items[i]['link']
                    jobplanet.loc[idx] = [company, link]
        except:
            company = kosdaq.회사명[idx]
            link = None
            jobplanet.loc[idx] = [company, link
    else: print("Error Code:" + rescode)
              
jobplanet

  0%|          | 0/1388 [00:00<?, ?it/s]

,회사명,링크
0,블리츠웨이,https://www.jobplanet.co.kr/companies/99313/re...
1,석경에이티,https://www.jobplanet.co.kr/companies/72848/in...
2,지놈앤컴퍼니,https://www.jobplanet.co.kr/companies/323397/s...
3,프리시젼바이오,https://www.jobplanet.co.kr/companies/346262/l...
4,알체라,https://www.jobplanet.co.kr/companies/346534/i...
...,...,...
1382,카스,https://www.jobplanet.co.kr/companies/54021/in...
1383,하이록코리아,https://www.jobplanet.co.kr/companies/3978/sal...
1384,SBI인베스트먼트,https://www.jobplanet.co.kr/companies/21009/sa...
1386,엠벤처투자,https://www.jobplanet.co.kr/companies/30208/in...


In [31]:
jobplanet['링크'] = 'https://www.jobplanet.co.kr/companies/' + jobplanet['링크'].str.split('/').str[4]+ '/reviews'

In [35]:
jobplanet.to_excel('jobplanet_link.xlsx', index=False)

## 3. 잡플래닛 리뷰 크롤링

In [ ]:
LOGIN_INFO ={'user[email]' : 'dnrl52@naver.com',
             'user[password]' : 'son1403***',
             'commit' : '로그인'}
session = requests.session()
res = session.post('https://www.jobplanet.co.kr/users/sign_in', data = LOGIN_INFO, verify=False)
res.raise_for_status()

result = []
for url in tqdm(jobplanet['링크']):
    print(url)
    for idx in range(1,6):
        time.sleep(0.1)
        params = f'?page={idx}'
        res = session.get(url+params)
        try: res.raise_for_status()
        except: continue
        soup = bs(res.text, 'lxml')
        cnt = 0
        try:            
            if soup.select('div.content_wrap'): 
                reviews = soup.select('div.content_wrap')
                for review in reviews:
                    txt1_tags = review.select('span.txt1')
                    date = txt1_tags[2].text
                    title = review.select_one('h2.us_label').text.replace('BEST','').strip()
                    tclist_tags = review.select('dl.tc_list')
                    contents = ' '.join([tclist.text.strip() for tclist in tclist_tags]).replace('\n','')
                    promotion = review.select('.bl_score')[0+cnt]['style'][6:-1]
                    welfare = review.select('.bl_score')[1+cnt]['style'][6:-1]
                    balance = review.select('.bl_score')[2+cnt]['style'][6:-1]
                    culture = review.select('.bl_score')[3+cnt]['style'][6:-1]
                    management = review.select('.bl_score')[4+cnt]['style'][6:-1]
                    review = [date, promotion, welfare, balance, culture, management, title, contents, url]
                    result.append(review)
                cnt += 5
            else:
                result.append([None, None, None, None, None, None, None, None, url])
        except:
            print("fail:" + str(idx))
            pass

jobplanet_review = pd.DataFrame(result, columns=['작성일','승진 기회 및 가능성','복지 및 급여','업무와 삶의 균형','사내문화','경영진','총평','리뷰','링크'])
jobplanet_review

  0%|          | 0/1274 [00:00<?, ?it/s]

https://www.jobplanet.co.kr/companies/99313/reviews
https://www.jobplanet.co.kr/companies/72848/reviews
https://www.jobplanet.co.kr/companies/323397/reviews
https://www.jobplanet.co.kr/companies/346262/reviews
https://www.jobplanet.co.kr/companies/346534/reviews
https://www.jobplanet.co.kr/companies/49064/reviews
https://www.jobplanet.co.kr/companies/89178/reviews
https://www.jobplanet.co.kr/companies/50778/reviews
https://www.jobplanet.co.kr/companies/243164/reviews
https://www.jobplanet.co.kr/companies/320910/reviews
fail:3
https://www.jobplanet.co.kr/companies/325498/reviews
fail:3
fail:4
fail:5
https://www.jobplanet.co.kr/companies/329748/reviews
https://www.jobplanet.co.kr/companies/50048/reviews
fail:5
https://www.jobplanet.co.kr/companies/212302/reviews
https://www.jobplanet.co.kr/companies/320531/reviews
https://www.jobplanet.co.kr/companies/11252/reviews
https://www.jobplanet.co.kr/companies/153618/reviews
https://www.jobplanet.co.kr/companies/141863/reviews
https://www.jobpla